<b> This code is designed to evaluated temperature with ENSO bins for the observational data of ELI </b>

In [1]:
import xarray as xr
from metpy.constants import g, omega as om, Re as a
from metpy.units import units
import netCDF4
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# Upload Observations seasonally average anomalies

dir = '/chinook/dallmann/Ob_data/Ob_data/'
fn_ob_anom = 'sst_anonDJFmean.nc'


data_temp = xr.open_dataset(dir+fn_ob_anom)

# Parse using MetPy, so we can use MetPy operations with it
#data_had_pr_his = data_had_pr_his.metpy.parse_cf()

# output directory and file
dro = '/chinook/dallmann/Ob_data/ELI_Bins/'
fn_output = 'sst_nina_strong_obs_anomDJFmean.nc'

In [3]:
# Upload the array for Nino_strong
dir = '/chinook/dallmann/ENSO_bins/'
fn_ENSO = 'Obs_Nina_strong_DJFmean.nc'

data_ENSO = xr.open_dataset(dir+fn_ENSO)
data_ENSO

<xarray.Dataset>
Dimensions:  (year: 8)
Coordinates:
  * year     (year) int64 1917 1934 1974 1989 1999 2000 2009 2011
Data variables:
    nino     (year) float32 ...

In [4]:
# Convert time to year so it can be found in an Integer64
# I am unsure if this is right since it already was seasonally averaged (unsure what to do about the .mean())
time = data_temp.groupby('time.year').mean()
# shift the temp data by one year because cdo reconizes the year assoicated with december. 
time = time.shift(year=1)

time

<xarray.Dataset>
Dimensions:      (lat: 89, lon: 180, nbnds: 2, year: 167)
Coordinates:
  * lon          (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * lat          (lat) float32 88.0 86.0 84.0 82.0 ... -82.0 -84.0 -86.0 -88.0
  * year         (year) int64 1853 1854 1855 1856 1857 ... 2016 2017 2018 2019
Dimensions without coordinates: nbnds
Data variables:
    time_bnds_2  (year, nbnds) float64 nan nan 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    sst          (year, lat, lon) float32 nan nan nan nan ... nan nan nan nan

In [5]:
ts_nina_strong = time.where(time['year'] == data_ENSO['year'], drop = True)
ts_nina_strong

<xarray.Dataset>
Dimensions:      (lat: 89, lon: 180, nbnds: 2, year: 8)
Coordinates:
  * year         (year) int64 1917 1934 1974 1989 1999 2000 2009 2011
  * lon          (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * lat          (lat) float32 88.0 86.0 84.0 82.0 ... -82.0 -84.0 -86.0 -88.0
Dimensions without coordinates: nbnds
Data variables:
    time_bnds_2  (year, nbnds) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    sst          (year, lat, lon) float32 -6.374e-05 -5.277e-05 ... nan nan

In [6]:
# Create a new netcdf file for preciptiation relative to this respective year
ts_nina_strong.to_netcdf(path=dro+fn_output)